In [ ]:
import accretion_disk_gpu
import numpy as np
import matplotlib.pyplot as plt
from numba import cuda

In [ ]:
# PARAMETRI FISSI
# 0. geometria BLR
inc = np.radians(30.)
xi_in = 200.
xi_out = 1800.
period = 120.

# 1. emissività
x_c = np.float32(300.0)
A = 84.
d = np.radians(134)
phi_0 = np.radians(24)
p = np.radians(45)
xi_sp = 870.

# 2. luminosità
period = 120.

# 3. specific intensity
lambda_emessa = 6563.
c = 3e5
sigma = np.float32(6563*1200/c)

#dimensioni per array di raggio, angolo, tempo e lunghezze d'onda
size_x = 175
size_phi = 175
size_time = 175
size_lam = 175

#creazione array (memoria CPU)
xi = np.linspace(xi_in, xi_out, size_x)
dr = xi[1]-xi[0]
xi = np.array(xi).astype(np.float32)
phi = np.linspace(0, 2*np.pi, size_phi)
phi = np.array(phi).astype(np.float32)
time = np.linspace(0, 150, size_time)
time = np.array(time).astype(np.float32)
lam = np.linspace(6200, 6900, size_lam)
lam = np.array(lam).astype(np.float32)

#creazione devices per salvare i risultati dei kernel CUDA
#1. Creo gli array necessari sulla memoria della CPU
area = np.zeros([size_lam, size_x,size_phi])
emissivity = np.zeros([size_lam, size_x,size_phi])
luminosity =np.zeros([size_lam, size_x,size_phi])
gauss = np.zeros([size_lam, size_x,size_phi])
flux = np.zeros([size_lam, size_x,size_phi])
flux_def = np.zeros([size_lam])
       
#2. Copio i device sulla memoria della GPU
d_xi = cuda.to_device(xi)
d_phi = cuda.to_device(phi)
d_time = cuda.to_device(time)
d_lam = cuda.to_device(lam)

#threads_per_block/blocks_per_grid per kernel area, emissività, luminosità e specific intensity, kernel prodotto
threads_per_block =  (4,4,4)
blocks_per_grid = (
      (d_lam.size + (threads_per_block[0] - 1)) // threads_per_block[0],
      (d_xi.size + (threads_per_block[1] - 1)) // threads_per_block[1],
      (d_phi.size + (threads_per_block[2] - 1)) // threads_per_block[2]
  )

#not used for now
threads_per_block_1 =  (4,4)
blocks_per_grid_1 = (
      (d_lam.size + (threads_per_block_1[0] - 1)) // threads_per_block_1[0],
      (d_time.size + (threads_per_block_1[1] - 1)) // threads_per_block_1[1]
  )

#threads_per_block/blocks_per_grid per kernel somma
threads_per_block_2 =  (4)
blocks_per_grid_2 = (
      (d_lam.size + (threads_per_block_1[0] - 1)) // threads_per_block_1[0],
      )